In [ ]:
from SALib.sample import latin, saltelli
from SALib.sample.morris import sample as morris_sample

from SALib.analyze import sobol
from SALib.analyze.morris import analyze as morris_analyze

from SALib.test_functions import Ishigami

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Here we create a wrapping interface that accepts the dummy parameter input (the 4th parameter `x4`), but strips the parameter values from the input array. This has the effect of making the fourth parameter inactive, and therefore insensitive

In [ ]:
# def ishigami_wrapper(param_values):
#     strip_dummy_param = param_values[:, 0:3]
#     return Ishigami.evaluate(strip_dummy_param)

In [ ]:
def oat_extremity_sample(problem):
    """Create extremity samples for a 'traditional' OAT analysis.
    
    Here we assume outputs will be affected at the extreme ends of the nominal bounds.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs        
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_vars = problem.get('num_vars')
    
    # nominal = np.array(nominal)
    experiment = [nominal[:]]
    
    # Samples represent runs with values at the extremes of each parameter boundary
    for i in range(num_vars):
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        after = i+1
        if i > 0:
            experiment.append(nominal[0:i] + [min_val] + nominal[after:])  # min
            experiment.append(nominal[0:i] + [max_val] + nominal[after:])  # max
        else:
            experiment.append([min_val] + nominal[after:])  # min
            experiment.append([max_val] + nominal[after:])  # max
            
    
    return np.array(experiment)
# End oat_extremity_sample()


def traditional_oat_sample(problem, N):
    """Create uniformly distributed samples for a 'traditional' OAT analysis.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * N : int, total number of samples to generate
    
    Returns
    ==========
    * np.array of samples
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_factors = int(problem.get('num_vars'))
    
    samples_per_param = N / num_factors
    
    if samples_per_param % 1 != 0:
        raise ValueError("Require equal number of samples per parameter. \
        {} samples across {} params is {}".format(N, num_factors, samples_per_param))
    
    samples_per_param = int(samples_per_param)
    
    # experiment = np.empty((N+1, num_factors))
    experiment = np.array(nominal * (N+1)).reshape(N + 1, num_factors)
    
    cols = np.arange(num_factors)
    
    # Samples represent runs with values at the extremes of each parameter boundary
    pos = 1
    for i in range(num_factors):
        
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        p_samp = np.linspace(min_val, max_val, samples_per_param)
        
        subset = slice(pos, pos+samples_per_param)
        experiment[subset, i] = p_samp
        
        pos += samples_per_param
    # End for
    
    return experiment
# End traditional_oat_sample()

    
def oat_analyze(problem, inputs, results):
    """Analyze a 'traditional' OAT result.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * inputs : np.array, samples used to generate results. 
               The first row is expected to hold nominal values.
    * results : np.array, model run results obtained using the samples.
                The first row is expected to hold nominal results.
                
    Returns
    ==========
    * np.array
    """
    X = inputs
    Y = results
    
    num_results, num_factors = X.shape
    nominal_values = problem.get('nominal')
    groupings = int((len(Y) - 1) / num_factors)

    # Set up result array (ignores one row as it holds the nominal results)
    si_res = np.full((groupings, num_factors), fill_value=np.nan)
    
    col = 0
    idx = 0
    for i in range(1, num_results):
        diff = (X[i, col] - X[0, col])
        if diff == 0.0:
            si_res[idx, col] = 0.0
        else:
            try:
                si_res[idx, col] = (Y[i] - Y[0]) / (X[i, col] - X[0, col])
            except IndexError:
                print(idx, col)
                raise
        # End if
        
        if i % groupings == 0:
            col += 1
            idx = 0
            continue
        # End if
        
        idx += 1
    # End for
    
    return si_res
# End oat_analyze()

Common variables and problem definitions

In [ ]:
SEED_VALUE = 101
SALTELLI_SAMPLES = 500
LATIN_SAMPLES = 1000
MORRIS_SAMPLES = 1000

SPEC_WO_INACTIVE = {
  'num_vars': 3,
  'names': ['x1', 'x2', 'x3'],
  'bounds': [[-np.pi, np.pi]]*3
}

SPEC_WITH_INACTIVE = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4
}